In [1]:
import os 

In [2]:
%pwd

'c:\\MLOps\\datascience_deployment_firstproject\\datascience_deployment\\research'

In [3]:
os.chdir("../") # go to my parent directory

In [4]:
%pwd

'c:\\MLOps\\datascience_deployment_firstproject\\datascience_deployment'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL:str 
    local_data_file: Path 
    unzip_dir: Path 

In [36]:
from src.datascience_deployment.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.datascience_deployment.utils.common import read_yaml,create_directories

import shutil

In [37]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config



In [38]:
## component - Data Ingestion 
import urllib.request as request 
from src.datascience_deployment import logger
import zipfile   
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config

    ## Download the zip file 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path:str 
        Extracts the zip file into data directory 
        Function returns none
        
        """
        unzip_path = self.config.unzip_dir
        zip_file = self.config.local_data_file

        os.makedirs(unzip_path,exist_ok=True)
        zip_file = os.path.join(self.config.local_data_file)

        try:
            shutil.unpack_archive(zip_file, unzip_path)
            print("Extraction successful!")
        except shutil.ReadError:
            print("Unknown format or corrupted file.")

In [39]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 

[2025-03-23 01:19:30,745]:INFO:common:yaml file: config\config.yaml loaded successfully:]
[2025-03-23 01:19:30,746]:INFO:common:yaml file: params.yaml loaded successfully:]
[2025-03-23 01:19:30,747]:INFO:common:yaml file: schema.yaml loaded successfully:]
[2025-03-23 01:19:30,748]:INFO:common:created directory at: artifacts:]
[2025-03-23 01:19:30,749]:INFO:common:created directory at: artifacts/data_ingestion:]
[2025-03-23 01:19:31,450]:INFO:25850806:artifacts/data_ingestion/data.zip download! with following info: 
 Server: GitHub.com
Date: Sat, 22 Mar 2025 19:49:31 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"7725cc40a7071dae5f3339642e99b085"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade


In [ ]:
# data_ingestion.extract_zip_file()

Extraction successful!
